# Precalculate FLORIS Solutions

Use FLASC tools to precalculate the FLORIS solutions across wind speeds and directions without wake steering to enable faster comparisons in later steps. The FLORIS solutions based on four different wake models are then saved in the "/precalculated_floris_solutions" folder. 

In [1]:
from pathlib import Path
from time import perf_counter as timerpc

import numpy as np

# from floris.tools import ParallelComputingInterface
from flasc.utilities import floris_tools as ftools
from flasc.utilities.utilities_examples import load_floris_smarteole as load_floris

In [2]:
# User settings
max_workers = 16
wake_models = ["jensen", "turbopark", "gch", "cc"]

In [3]:
# Precalculate FLORIS solutions
root_path = Path.cwd() / "precalculated_floris_solutions"
root_path.mkdir(exist_ok=True)

for wake_model in wake_models:
    fn = root_path / "df_fi_approx_{:s}.ftr".format(wake_model)
    if fn.is_file():
        print("FLORIS table for '{:s}' model exists. Skipping...".format(wake_model))
        continue

    start_time = timerpc()
    print("Precalculating FLORIS table for '{:s}' model...".format(wake_model))
    fm, _ = load_floris(wake_model=wake_model)
    # fi_pci = ParallelComputingInterface(
    #     fi=fi,
    #     max_workers=max_workers,
    #     n_wind_direction_splits=max_workers,
    #     print_timings=True,
    # )
    df_fm_approx = ftools.calc_floris_approx_table(
        fm=fm,  # fi_pci,
        wd_array=np.arange(0.0, 360.01, 3.0),
        ws_array=np.arange(1.0, 30.01, 1.0),
        ti_array=[0.03, 0.06, 0.09, 0.12, 0.15],
    )
    end_time = timerpc()
    print("Computation time: {:.2f} s".format(end_time - start_time))
    df_fm_approx.to_feather(fn)

Precalculating FLORIS table for 'jensen' model...
Generating a df_approx table of FLORIS solutions covering a total of 18150 cases.
Finished calculating the FLORIS solutions for the dataframe.
Computation time: 3.92 s
Precalculating FLORIS table for 'turbopark' model...
Generating a df_approx table of FLORIS solutions covering a total of 18150 cases.
Finished calculating the FLORIS solutions for the dataframe.
Computation time: 6.61 s
Precalculating FLORIS table for 'gch' model...
Generating a df_approx table of FLORIS solutions covering a total of 18150 cases.
Finished calculating the FLORIS solutions for the dataframe.
Computation time: 5.15 s
Precalculating FLORIS table for 'cc' model...
Generating a df_approx table of FLORIS solutions covering a total of 18150 cases.
Finished calculating the FLORIS solutions for the dataframe.
Computation time: 6.29 s
